#### Import dependencies and configure globals
This example, shows how to use gevopy to evolve a simple Haploid genotype in order to obtain a phenotype with at least 10 ones in it's chromosome. In the process, all the phenotypes are stored inside a neo4j database.

To do so, it is required to import gevopy and generate the Genotype, Fitness and Algorithm required. For the aim of simplicity, we will use some already defined examples from this library. Additionally it is required to import the class Neo4jInterface to specify the connection with our database.

Additionally, you can use neo4j library to verify the connection and the logging module to increase the verbosity of the evolution process.

In [1]:
import logging

from neo4j import GraphDatabase

import gevopy as ea
from examples.algorimths import BasicPonderated as MyAlgorithm
from examples.evaluation import MostOnes as MyFitness
from examples.genotypes import Bacteria as MyGenotype
from gevopy.database import Neo4jInterface


In [2]:
LOG_FORMAT = "%(asctime)s - %(name)s - %(levelname)s\n  %(message)s"
logging.basicConfig(level="INFO", format=LOG_FORMAT)
logger = logging.getLogger("gevopy.Experiment")
logger.setLevel(logging.INFO)  # Only Experiment INFO


#### Verify the database connection
Connect to a database by creating a neo4j Driver object and providing a URL and an authentication token. Once you have a Driver instance, use the .verify_connectivity() method to ensure that a working connection can be established.


In [3]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "bolt:"  # "<URI for Neo4j database>"
AUTH = ""  # ("<Username>", "<Password>")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()


### Create and run the experiment
Execute an experiment by creating an instance and assigning the argument database with the driver. The experiment will automaticaly manage sessions and use the methods .execute_read and .execute_write.

Use the experiment session as a normal example to evolve a Genotype. As minimalist example, add some phenotypes as a base and start the evolution by using .run method.

In [4]:
experiment = ea.Experiment(
    fitness=MyFitness(cache=True, scheduler="synchronous"),
    algorithm=MyAlgorithm(survival_rate=0.2),
    database=Neo4jInterface(URI, auth=AUTH),
)

with experiment.session() as session:
    session.add_phenotypes([MyGenotype() for _ in range(12)])
    statistics = session.run(max_score=10)


2022-12-15 08:51:44,230 - gevopy.Experiment - INFO
  [3c64603d-ed14-4b72-9fd3-ab67e3c3f783]: [gen:0]: Start of evolutionary experiment execution
2022-12-15 08:51:44,267 - gevopy.Experiment - INFO
  [3c64603d-ed14-4b72-9fd3-ab67e3c3f783]: [gen:1]: Completed cycle; 9
2022-12-15 08:51:44,286 - gevopy.Experiment - INFO
  [3c64603d-ed14-4b72-9fd3-ab67e3c3f783]: [gen:2]: Completed cycle; 9
2022-12-15 08:51:44,309 - gevopy.Experiment - INFO
  [3c64603d-ed14-4b72-9fd3-ab67e3c3f783]: [gen:3]: Completed cycle; 10
2022-12-15 08:51:44,310 - gevopy.Experiment - INFO
  [3c64603d-ed14-4b72-9fd3-ab67e3c3f783]: [gen:3]: Experiment execution completed successfully


In [5]:
statistics


Evolutionary algorithm execution report:
  Executed generations: 3
  Best phenotype: 29f25f6b-2363-4865-8763-829ec4b28ce8
  Best score: 10

### Close connections
Unless you created them using the with statement, call the .close method on all Driver and Session instances out of the experiment to release any resources still held by them.

In [6]:
experiment.close()
